### Capstone Project:  The Battle of Neighborhoods (Week 1)

Now that you have been equipped with the skills and the tools to use location data to explore a geographical location, over the course of two weeks, you will have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice or to come up with a problem that you can use the Foursquare location data to solve. If you cannot think of an idea or a problem, here are some ideas to get you started:

1. In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. Is New York City more like Toronto or Paris or some other multicultural city? I will leave it to you to refine this idea.

2. In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?

These are just a couple of many ideas and problems that can be solved using location data in addition to other datasets. No matter what you decide to do, make sure to provide sufficient justification of why you think what you want to do or solve is important and why would a client or a group of people be interested in your project.

Review criterialess 
This capstone project will be graded by your peers. This capstone project is worth 70% of your total grade. The project will be completed over the course of 2 weeks. Week 1 submissions will be worth 30% whereas week 2 submissions will be worth 40% of your total grade.

For this week, you will required to submit the following:

1. A description of the problem and a discussion of the background. (15 marks)
2. A description of the data and how it will be used to solve the problem. (15 marks)

For the second week, the final deliverables of the project will be:

1. A link to your Notebook on your Github repository, showing your code. (15 marks)
2. A full report consisting of all of the following components (15 marks):

Introduction where you discuss the business problem and who would be interested in this project.
Data where you describe the data that will be used to solve the problem and the source of the data.
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
Results section where you discuss the results.
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
Conclusion section where you conclude the report.

3. Your choice of a presentation or blogpost. (10 marks)

### Create Toronto Dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
rows = soup.find_all('tr')
for row in rows:
    row_td = row.find_all('td')

str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()

import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)

df = pd.DataFrame(list_rows)
df1 = df[0].str.split(',', expand=True)

# Remove '[',']' on Column 0 and Column 1
df1[0] = df1[0].str.strip('[')
df1[2] = df1[2].str.strip(']')

# Remove \n on first 3 columns
for i in range(3):
    df1[i] = df1[i].str.strip('\n')

# Keep first three columns
df2 = df1.iloc[:, : 3]

# Get table headers
col_labels = soup.find_all('th')
all_header = []
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
all_header.append(cleantext2)

# Replace headers
columns = {0:'Postal code', 1:'Borough', 2:'Neighborhood'}
df3 = df2.rename(columns=columns)

# Remove rows contain 'Not assigned','NS','NL','B', and 'None' in Borough column
df4 = df3[~df3['Borough'].str.contains("Not assigned|NS|NL|B", na=True)]

# Remove rows contain 'None' in Neighborhood column
df5 = df4[~df4['Neighborhood'].str.contains("N/A", na=True)]

# Replace / with , in Neighborhood column
df5['Neighborhood'] = df5['Neighborhood'].str.replace(' /',',')

df6 = df5[df5['Postal code'].str.contains("M5A", na=True)]

df5.shape

df_geocoder = pd.read_csv("Geospatial_Coordinates.csv")

df_toronto = pd.merge(df5, df_geocoder, on='Postal code', how='outer')
df_toronto.head(5)

/Users/richardxue/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Use Foursquare API to explore the neighborhoods and segment them

In [5]:
# My Foursquare credential
VERSION = '20180605' 
CLIENT_ID='0QUMVQD2MXECCBFR4U3CDLQFO2WVVQOE5JGGBR04VJJ5Q5OO'
CLIENT_SECRET = 'V2RDAFWOSJ4IKWKLNNAIZDEE30EDGSA5FE0LTROSJAVGYHFQ'

#### Explore the second neighborhood in Toronto dataframe

In [7]:
df_toronto[1:2]

,Postal code,Borough,Neighborhood,Latitude,Longitude
1,M4A,North York,Victoria Village,43.725882,-79.315572


#### Get the 2nd neighborhood's name, latitude and longitude values

In [8]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] 
neighborhood_longitude = df_toronto.loc[0, 'Longitude']
neighborhood_name = df_toronto.loc[0, 'Neighborhood']

#### Get the top 100 venues that are in Victoria Village within a radius of 400 meters

In [26]:
num_venues = 100
radius = 400 
URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, num_venues)
URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0QUMVQD2MXECCBFR4U3CDLQFO2WVVQOE5JGGBR04VJJ5Q5OO&client_secret=V2RDAFWOSJ4IKWKLNNAIZDEE30EDGSA5FE0LTROSJAVGYHFQ&v=20180605&ll=43.7532586,-79.3296565&radius=400&limit=100'

#### Make HTTP GET Request via Foursquare API

In [27]:
import requests
response = requests.get(URL).json()
response

{'meta': {'code': 200, 'requestId': '5e842a6895feaf001b74605b'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.75685860360001,
    'lng': -79.32468189187942},
   'sw': {'lat': 43.749658596399996, 'lng': -79.33463110812058}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLng

The information from the above response is in the items key. We need the following get_category_type() function from the Foursquare lab to transform the JSON data.

In [28]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Transform JSON Respon into a Pandas Dataframe

In [29]:
from pandas.io.json import json_normalize

venues = response['response']['groups'][0]['items']

# Normalize Json data
nearby_venues = json_normalize(venues)

# Filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# Filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# Clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,649 Variety,Convenience Store,43.754513,-79.331942
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [30]:
print('There are {} venues identified by Foursquare.'.format(nearby_venues.shape[0]))

There are 3 venues identified by Foursquare.


### Explore All Neighborhoods in Toronto

Define a function to repeat the above process to all neighborhoods in Toronto

In [37]:
def findNearbyVenues(names, latitudes, longitudes, radius=400):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        #  API GET Request URL
        URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, num_venues)
            
        # Make the GET request
        response = requests.get(URL).json()["response"]['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], 
            v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in response])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 
                'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

#### Use the above function to find nearby venues for all neighborhoods and create a new dataframe called df_toronto_venues.

In [38]:
df_toronto_venues = findNearbyVenues(names=df_toronto['Neighborhood'], latitudes=df_toronto['Latitude'],
                    longitudes=df_toronto['Longitude'])

 Parkwoods
 Victoria Village
 Regent Park, Harbourfront
 Lawrence Manor, Lawrence Heights
 Queen's Park, Ontario Provincial Government
 Islington Avenue
 Malvern, Rouge
 Don Mills
 Parkview Hill, Woodbine Gardens
 Garden District
 Glencairn
 West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
 Rouge Hill, Port Union, Highland Creek
 Don Mills
 Woodbine Heights
 St. James Town
 Humewood-Cedarvale
 Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
 Guildwood, Morningside, West Hill
 The Beaches
 Berczy Park
 Caledonia-Fairbanks
 Woburn
 Leaside
 Central Bay Street
 Christie
 Cedarbrae
 Hillcrest Village
 Bathurst Manor, Wilson Heights, Downsview North
 Thorncliffe Park
 Richmond, Adelaide, King
 Dufferin, Dovercourt Village
 Scarborough Village
 Fairview, Henry Farm, Oriole
 Northwood Park, York University
 East Toronto
 Harbourfront East, Union Station, Toronto Islands
 Little Portugal, Trinity
 Kennedy Park, Ionview, East Birchmount Park
 Bayview Village


In [39]:
df_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Brookbanks Park,43.751976,-79.332140,Park
3,Victoria Village,43.725882,-79.315572,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,"Regent Park, Harbourfront",43.654260,-79.360636,Brookbanks Park,43.751976,-79.332140,Park


In [41]:
print(df_toronto_venues.shape)

(251, 7)


#### Find out the Unique Venue Category

In [42]:
df_toronto_venues['Venue Category'].unique()

array(['Park', 'Food & Drink Shop', 'Convenience Store'], dtype=object)